# FDH as a sign-constrained variant of the ICNLS problem

- Author: Sheng Dai (sheng.dai@aalto.fi)
- Date  : June 18, 2020

Reference:

[1] Keshvari A, Kuosmanen T. Stochastic non-convex envelopment of data: Applying isotonic regression to frontier estimation. Eur J Oper Res 2013;231:481–91.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import FDH2ICNLS

In [3]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)

In [4]:
# output
y = df['Energy']

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

In [5]:
# define and solve the DEA radial model

model = FDH2ICNLS.fdh2icnls(y, x)

# using local solver (MOSEK API)
from pyomo.opt import SolverFactory
opt = SolverFactory("mosek")
results = opt.solve(model, tee=True)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 3714            
  Cones                  : 0               
  Scalar variables       : 356             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 35
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0              

In [6]:
# retrive the residuals (additive FDH efficiency measure)
val = list(model.e[:].value)
eps = np.asarray(val)

In [7]:
# multiplicative DEA efficiency measure (c.f., DEA_oo_vrs)
theta = 1 - eps/y

In [8]:
theta

0     1.640003
1     1.983872
2     1.666667
3     1.875549
4     1.481494
        ...   
84    1.741936
85    1.000004
86    1.000004
87    1.466719
88    1.238096
Name: Energy, Length: 89, dtype: float64

**Note**: In the single-output setting, the additive FDH efficiency measure (i.e., this case) is equivalent to the multiplicative FDH efficiency measure (i.e., [Radial output oriented FDH model](https://nbviewer.jupyter.org/github/ds2010/pyStoNED-Tutorials/blob/master/FDH/FDH_oo.ipynb)) in the sense that:
$\theta_i^{DEA} = 1 - eps_i^{DEA}/y_i$